In [1]:
# Install Whisper + dependencies
!pip install git+https://github.com/openai/whisper.git
!sudo apt update && sudo apt install ffmpeg -y



  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-30o_p_gk
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-30o_p_gk
  Resolved https://github.com/openai/whisper.git to commit c0d2f624c09dc18e709e37c2ad90c039a4eb72a2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=f0929d102567499e55760a9c312d9915c34107f465e7c40fe057628121cee0c7
  Stored in directory: /tmp/pip-ephem-wheel-cache-pdroe3bp/wheels/c3/03/25/5e0ba78bc27a3a089f137c9f1d92fdfce16d06996c071a016c
Successfully built openai-whisper
Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,628 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InR

In [2]:
# Import libraries
import whisper

# Load pre-trained Whisper model
# Options: tiny, base, small, medium, large
model = whisper.load_model("small")

# Upload an audio file (wav, mp3, m4a etc.)
from google.colab import files
uploaded = files.upload()

# Take first uploaded file
audio_file = list(uploaded.keys())[0]

# Transcribe the audio
result = model.transcribe(audio_file)

# Print the transcription
print("Transcription:\n", result["text"])


100%|███████████████████████████████████████| 461M/461M [00:29<00:00, 16.6MiB/s]


Saving clean.opus to clean.opus
Transcription:
  Hello everyone, I am Siddhas form Patram, BTEC third year studying at ITVLI TSI


In [3]:
from google.colab import files
uploaded = files.upload()

# Take first uploaded file
audio_file = list(uploaded.keys())[0]

# Transcribe the audio
result = model.transcribe(audio_file)

# Print the transcription
print("Transcription:\n", result["text"])

Saving nosiy.opus to nosiy.opus
Transcription:
  इसलग man음 वी तरकतम constructing �odi


In [6]:
!pip install noisereduce


In [9]:
import noisereduce as nr
from scipy.io import wavfile
# 1. Load the noisy audio file and its sample rate
rate, data = wavfile.read("nosiy.wav")

# 2. Select a sample of pure noise
# (e.g., the first 1 second). This part is crucial!
noise_clip = data[0:rate]
# 3. Perform noise reduction
reduced_noise_data = nr.reduce_noise(y=data.astype(float), sr=rate,y_noise=noise_clip)
# 4. Save the clean audio to a new file
wavfile.write("my_clean_audio.wav", rate,
reduced_noise_data)
print("Noise reduction complete!")



Noise reduction complete!


In [12]:
# Give the path to your audio file
audio_file = "my_clean_audio.wav"   # change this to your file path

# Transcribe the audio
result = model.transcribe(audio_file)

# Print the transcription
print("Transcription:\n", result["text"])


Transcription:
  Hello everyone, I am Chris Walfa-Grath, setting a diet in Gleinfunk, DSI is in France.


In [26]:
lis=[ "clean.opus","nosiy.wav","my_clean_audio.wav",]
results=[]
for i in lis:

    # Transcribe the audio
    result = model.transcribe(i)
    results.append(result)
    # Print the transcription
    print("Transcription:\n", result["text"])


Transcription:
  Hello everyone, I am Siddhas form Patram, BTEC third year studying at ITVLI TSI
Transcription:
  प्रद़ा मेप स्लने के चलने ने ठिन refugee camp यह झोथा म मन पिर मजब. हम आधियश AhíAME मैं negotiating with, यह आप मेफ़नी श्�ax slippers यह्तर donner यहानकाी ठाई आप मैं. फीरнакоतल कब काहि लेकप क अड़ चेण चुर कर करन्ु. पुरकाक Faraya �しまगया यहगा Oğlum fahren बाँछ
Transcription:
  Hello everyone, I am Chris Walfa-Grath, setting a diet in Gleinfunk, DSI is in France.


In [20]:
!pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 39.0 MB/s eta 0:00:00


In [28]:
from jiwer import wer

# Ground truth (what you actually said)
reference = "Hello everyone, I am Siddhas form Patram, BTEC third year studying at ITVLI TSI"

# If results is a list of lists or dicts, flatten it
# Example: [["hello everyone i am ..."]] → "hello everyone i am ..."
#          [{"text": "hello everyone i am ..."}] → "hello everyone i am ..."
hypotheses = []
for r in results:
    if isinstance(r, dict) and "text" in r:
        hypotheses.append(r["text"])
    elif isinstance(r, list):
        hypotheses.append(" ".join(r))
    else:
        hypotheses.append(str(r))

# Now compute WER for each hypothesis
for i, hyp in enumerate(hypotheses):
    error = wer(reference, hyp)
    print(f"File {i+1}:")
    print(f"  Reference   : {reference}")
    print(f"  Hypothesis  : {hyp}")
    print(f"  WER         : {error:.2f}\n")



File 1:
  Reference   : Hello everyone, I am Siddhas form Patram, BTEC third year studying at ITVLI TSI
  Hypothesis  :  Hello everyone, I am Siddhas form Patram, BTEC third year studying at ITVLI TSI
  WER         : 0.00

File 2:
  Reference   : Hello everyone, I am Siddhas form Patram, BTEC third year studying at ITVLI TSI
  Hypothesis  :  प्रद़ा मेप स्लने के चलने ने ठिन refugee camp यह झोथा म मन पिर मजब. हम आधियश AhíAME मैं negotiating with, यह आप मेफ़नी श्�ax slippers यह्तर donner यहानकाी ठाई आप मैं. फीरнакоतल कब काहि लेकप क अड़ चेण चुर कर करन्ु. पुरकाक Faraya �しまगया यहगा Oğlum fahren बाँछ
  WER         : 3.50

File 3:
  Reference   : Hello everyone, I am Siddhas form Patram, BTEC third year studying at ITVLI TSI
  Hypothesis  :  Hello everyone, I am Chris Walfa-Grath, setting a diet in Gleinfunk, DSI is in France.
  WER         : 0.79

